In [ ]:
import numpy as np 
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
train_df = pd.read_csv('../input/train.csv')
test_df = pd.read_csv('../input/test.csv')

In [ ]:
X_train = train_df.iloc[:, 1:].values.reshape(-1, 28, 28, 1)
y_train = train_df.iloc[:, 0].values
X_test = test_df.values.reshape(-1, 28, 28, 1)

In [ ]:
index = 100
plt.imshow(X_train[index].reshape(28,28))
print(y_train[index])

In [ ]:
def to_1hot(labels):
    ones = tf.one_hot(labels, 10, axis = 0)
    with tf.Session() as sess:
        ones = sess.run(ones)
    return ones
y_train = to_1hot(y_train).T

In [ ]:
X_train = X_train / 255
X_test = X_test / 255

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(8, kernel_size = (5, 5), strides = (1, 1), activation = 'relu'))
model.add(tf.keras.layers.Conv2D(32, kernel_size = (5, 5), strides = (1, 1), padding = 'same', activation = 'relu'))
model.add(tf.keras.layers.AvgPool2D(pool_size = (2, 2), strides = (2, 2), padding = 'same'))
model.add(tf.keras.layers.AvgPool2D(pool_size = (2, 2), strides = (2, 2), padding = 'same'))
model.add(tf.keras.layers.Conv2D(64, kernel_size = (5, 5), strides = (1, 1), padding = 'same', activation = 'relu'))
model.add(tf.keras.layers.Conv2D(32, kernel_size = (5, 5), strides = (1, 1), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.2, seed  = 1))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation = "softmax"))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
earlyStopping = EarlyStopping(patience=10, verbose=0)
mcp_save = ModelCheckpoint('digit.h5', verbose=0, save_best_only=True, save_weights_only=True)
reduce_lr_loss = ReduceLROnPlateau(factor=0.1, patience=5, min_lr=0.000001, verbose=0)
callbacks = [earlyStopping, mcp_save, reduce_lr_loss]

In [ ]:
result = model.fit(X_train, y_train, batch_size=64, validation_split = 0.2, callbacks = callbacks, epochs = 40, verbose = 1)

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Learning curve")
plt.plot(result.history["loss"], label="loss")
plt.plot(result.history["val_loss"], label="val_loss")
plt.plot( np.argmin(result.history["val_loss"]), np.min(result.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

In [ ]:
import random

In [ ]:
index = random.randint(0, len(X_test))
print('index = ', index)
plt.imshow(X_test[index].reshape(28,28))

In [ ]:
pred = model.predict(X_test[index:index+1])

In [ ]:
max = np.argmax(pred)
print('this number is', max)

In [ ]:
prediction = model.predict(X_test)

In [ ]:
prediction = np.argmax(prediction, axis=1)

In [ ]:
submission = pd.DataFrame({'ImageId':np.arange(1, X_test.shape[0] + 1), 
                          'Label': prediction})

In [ ]:
from IPython.display import HTML
import pandas as pd
import numpy as np
import base64

# function that takes in a dataframe and creates a text link to  
# download it (will only work for files < 2MB or so)
def create_download_link(df, title = "Download CSV file", filename = "forest.csv"):  
    csv = df.to_csv()
    b64 = base64.b64encode(csv.encode())
    payload = b64.decode()
    html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
    html = html.format(payload=payload,title=title,filename=filename)
    return HTML(html)

# create a random sample dataframe
# create a link to download the dataframe
create_download_link(submission)